In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
import numpy 
import pandas as pd
import pickle as pkl
import dataframe

In [14]:
'''df = pd.read_csv('data/tweets_ats.csv')
tweets = df['renderedContent'].tolist()'''

In [5]:
app_store = pkl.load(open("./data/app_store.pkl", "rb"))
app_store = pd.json_normalize(app_store)

play_store = pkl.load(open("./data/play_store.pkl", "rb"))
play_store = pd.json_normalize(play_store)
df = dataframe.GetDataframeStores(play_store,app_store)

c:\Users\Si Ben Tran\Documents\GitHub\MaKEathonPostFinance\dataframe.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_app["content"] = df_app["title"] + ": " + df_app["review"]


In [8]:
df["lencontent"] = df["content"].apply(lambda x : len(x))
df = df[df["lencontent"] < 514]
df = df[df["language_prob"] == "en"]
tweets = df["content"].tolist()
tweets

["It was better before: In general I like the app, but it was much better before when I could see the balance of all my accounts at once in the main screen. Please bring Theo's feature back!",
 'Poor limited functionality: The previous app was good - you could do and see everything needed. The ‘upgrade’ with the new ‘features’ is very limited - seems the UX design manager prefers logging into websites and thinks we should all do the same. This is now only a simple payment and display transactions app, nothing more. Bad decision that I’m sure has lost a lot of customers, myself included.',
 'Horrible: The previous one was super. This is horrible. Unreliable. UX nightmare. Features are missing. Features are present and not working.',
 'Howto make a good app become the worst ever: A very good example for ux/ui school about how to destroy an app.\n\nEverything good have been changed to the worse which is somehow quiet an impressive performance.',
 'The worst bank and app of CH: I can’t tel

In [9]:
# load model and tokenizer in English
roberta = "cardiffnlp/twitter-roberta-base-sentiment"

model = AutoModelForSequenceClassification.from_pretrained(roberta)
tokenizer = AutoTokenizer.from_pretrained(roberta)

labels = ['Negative', 'Neutral', 'Positive']

Downloading:   0%|          | 0.00/747 [00:00<?, ?B/s]

c:\Users\Si Ben Tran\daw\lib\site-packages\huggingface_hub\file_download.py:123: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Si Ben Tran\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Downloading:   0%|          | 0.00/499M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [10]:
# Clustering the Sentences
positive = []
negative = []
neutral = []

count = 0

for tweet in tweets:
    count += 1
    print(count)
    tweet_words = []
    for word in tweet.split(' '):
        if word.startswith('@') and len(word) > 1:
            word = '@user'
    
        elif word.startswith('http'):
            word = "http"
        tweet_words.append(word)

    tweet_proc = " ".join(tweet_words)

    encoded_tweet = tokenizer(tweet_proc, return_tensors='pt')

    output = model(**encoded_tweet)

    scores = output[0][0].detach().numpy()
    scores = softmax(scores)

    positive.append(scores[0])
    negative.append(scores[1])
    neutral.append(scores[2])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [12]:
# create a new dataframe with the sentiment scores
df_sentiment = pd.DataFrame({'positive': positive, 'negative': negative, 'neutral': neutral})
df_sentiment

,positive,negative,neutral
0,0.007042,0.042536,0.950422
1,0.769134,0.191246,0.039620
2,0.960920,0.035153,0.003927
3,0.454550,0.335977,0.209473
4,0.976221,0.019958,0.003821
...,...,...,...
516,0.207136,0.659662,0.133202
517,0.007061,0.124166,0.868773
518,0.909631,0.082987,0.007382
519,0.003112,0.070366,0.926522


In [14]:
df_test = pd.concat([df["content"], df_sentiment], axis=1)

InvalidIndexError: Reindexing only valid with uniquely valued Index objects